<a href="https://colab.research.google.com/github/heinohen/Textual-Data-Analysis/blob/main/TDA_exercise11_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Textual data analysis exercise 11 - basic chatbot



## Setup

### installs

In [1]:
%%bash

pip3 install -q transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.0 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Fri Feb 21 15:54:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   38C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### imports

In [3]:
import transformers

from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login
from google.colab import userdata
import datetime

In [4]:
hf_token = userdata.get('huggingface')
login(token=hf_token, add_to_git_credential=True)

## model

In [5]:
MODEL_NAME = 'HuggingFaceTB/SmolLM2-1.7B-Instruct'

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
device = "cuda"
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

# TASK

## description

* At least 5 questions about basic facts about the world (e.g. the capital of a country)
* At least 5 arithmetic questions ranging from trivial ("what is 1+1?") to more complex
* Inform the system of a secret word (e.g. "zebra"), then after a few other questions ask it what the secret word is. Make sure you understand where the memory of that secret word is.

# PROGRAM

## data storage

In [11]:
message_history = []

## actual functionality

In [12]:
while True:
    user_input = input('Say something ("exit" to quit): ')
    if user_input == 'exit':
        break
    message_history.append({"role": "user", "content": f"{user_input}"})
    input_text=tokenizer.apply_chat_template(message_history, tokenize=False)
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
    outputs = model.generate(inputs,
                             max_new_tokens=50,
                             temperature=0.2,
                             top_p=0.9,
                             do_sample=True)
    result = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True).rstrip().split() # this starts to decode the answer from the end of the current length of the previous messages and splits it to list
    answer = " ".join(result[1:]) # removes the "assistant" from the beginning
    message_history.append({"role": "assistant", "content": answer})
    print(answer)

Say something ("exit" to quit): hello!
Hello! How can I assist you today?
Say something ("exit" to quit): what are you
I'm SmolLM, a small language model designed to provide information and answer questions to the best of my ability. I'm here to help with a wide range of topics, from general knowledge to specific inquiries.
Say something ("exit" to quit): ok let's start
I'm ready to help. What's your first question or topic you'd like to discuss?
Say something ("exit" to quit): what is the capital of finland
The capital of Finland is Helsinki. It's a beautiful city located on the Gulf of Finland, known for its stunning architecture, vibrant cultural scene, and delicious food.
Say something ("exit" to quit): Who is the president of finland
The President of Finland is Sauli Niinistö. He has been the President of Finland since 2012 and is a member of the conservative National Coalition Party.
Say something ("exit" to quit): Are you sure, didn't finland have elections in 2024
Yes, you are 

# MESSAGE LOG

In [13]:
length = 10

for m in message_history:

  message = f"{m['role']}"
  print(f"{message:<{length}}","=>\t", m['content'])

user       =>	 hello!
assistant  =>	 Hello! How can I assist you today?
user       =>	 what are you
assistant  =>	 I'm SmolLM, a small language model designed to provide information and answer questions to the best of my ability. I'm here to help with a wide range of topics, from general knowledge to specific inquiries.
user       =>	 ok let's start
assistant  =>	 I'm ready to help. What's your first question or topic you'd like to discuss?
user       =>	 what is the capital of finland
assistant  =>	 The capital of Finland is Helsinki. It's a beautiful city located on the Gulf of Finland, known for its stunning architecture, vibrant cultural scene, and delicious food.
user       =>	 Who is the president of finland
assistant  =>	 The President of Finland is Sauli Niinistö. He has been the President of Finland since 2012 and is a member of the conservative National Coalition Party.
user       =>	 Are you sure, didn't finland have elections in 2024
assistant  =>	 Yes, you are correct. Fin

In very brief, what did you think about the capabilities of the chatbot you created? Of the factors impacting LLM performance that we discussed on the lecture, which one (or ones) do you think are primarily limiting its performance?

* well at least it is confident about the answers, not all were correct but still
* of the factors impacting LLM perf i would say that the quantity of FLOPS used for creation of the model
* the secret word is actually stored in the `message_history`